In [174]:
import cv2 
import os

In [177]:
# Load Images
Directories = ['Set5', 'Set14', 'Urban100', 'BSD100', 'Celeba_HQ']
scales = [2,3,4,8]

def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        img = Image.open(folder + "/" + filename)
        if img is not None:
            images.append(img)
    return images

In [21]:
# Downsample
from PIL import Image 
import PIL 

def downsample(): 
    for directory in Directories:
        images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + '/HR')
        for scale in scales: 
            disk_dir = '/Users/ege/Desktop/DataSets/' + directory + "/LR/" + str(scale) + "_b/"
            for i, oriimg in enumerate(images): 
                width, height = oriimg.size
                #img = oriimg.resize((int(width/scale),int(height/scale)), Image.BICUBIC)
                #img = cv2.resize(oriimg,None, fx=1/scale, fy=1/scale, interpolation = cv2.INTER_CUBIC)

                img.save(disk_dir +str(i)+ ".png","PNG")
                print(img.size)

In [178]:
from sewar.full_ref import mse, rmse, psnr, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb 

def evaluate(img1,img2):
    print(ssim(img1,img2))
    

In [179]:
# Upsample
# Load Images 

Directories = ['Set14']
scales = [2,3,4,8]

def upsample():
    for directory in Directories:
        for scale in scales: 
            images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/LR/" + str(scale)) 
            disk_dir = '/Users/ege/Desktop/DataSets/' + directory + "/HR/" + str(scale) + "_upscaled/"
            for i, oriimg in enumerate(images): 
                width, height = oriimg.size
                img = oriimg.resize((int(width*scale),int(height*scale)), Image.BICUBIC)
                #img = cv2.resize(oriimg,None, fx=scale, fy=scale, interpolation = cv2.INTER_CUBIC)
                img.save(disk_dir +str(i)+ ".png","PNG")
            

In [180]:
import numpy 
import math
import cv2

def psnr_numpy(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [137]:
# Read and evaluate
Directories = ['Set14']
scales = [4]

import numpy as np
import pandas as pd

from sewar.full_ref import mse, rmse, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb, psnr

metric_name = []
upsampling_method = []
downsampling_method = []
image_set = []
image_scale = [] 
image_number = []
metric_1 = []
metric_2 = []
metric_3 = []
metric_4 = []
metric_5 = []
metric_6 = []
metric_7 = []
metric_8 = []

for directory in Directories:    
    for scale in scales: 
        images_1 = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + "Original") 
        images_2 =  load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + str(scale) + "_upscaled/")

        #images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + '/HR')
        for i, oriimg in enumerate(images_1): 
            #evaluated_value = ssim(oriimg,images_2[i]) #evaluate(oriimg,images_2[i])
            #value_mse = uqi(images_1[i],images_2[i])
            #print(value_mse)
            
            pil_image_1 = images_1[i].convert('RGB') 
            pil_image_2 = images_2[i].convert('RGB')
            
            open_cv_image_1 = np.array(pil_image_1) 
            open_cv_image_2 = np.array(pil_image_2) 
            
            # Convert RGB to BGR 
            open_cv_image_1 = open_cv_image_1[:, :, ::-1].copy()
            open_cv_image_2 = open_cv_image_2[:, :, ::-1].copy()
            
            try:
                mse_value = mse(open_cv_image_1,open_cv_image_2) #evaluate(oriimg,images_2[i])
                rmse_value = rmse(open_cv_image_1,open_cv_image_2)
                psnr_value = psnr(open_cv_image_1,open_cv_image_2)
                rmse_sw_value = psnr_numpy(open_cv_image_1,open_cv_image_2)
                ssim_value = ssim(open_cv_image_1,open_cv_image_2)
                sam_value = sam(open_cv_image_1,open_cv_image_2)
                msssim_value = msssim(open_cv_image_1,open_cv_image_2)
                psnrb_value = psnrb(open_cv_image_1,open_cv_image_2) 
                
            except:
                mse_value = None
                rmse_value = None
                psnr_value = None
                ssim_value = None
                sam_value = None
                msssim_value = None
                psnrb_value = None
                
                print('Folder:' + str(scale) + '_upscaled image number:' + str(i))
                pass
            

            # Merge all the lists/Pandas Dataframes in one and then concatenate them as a LIST
            upsampling_method.append('bicubic')
            downsampling_method.append('bicubic')
            image_set.append(str(directory))
            image_scale.append(str(scale))
            image_number.append(str(i))
            metric_1.append(mse_value)
            metric_2.append(rmse_value)
            metric_3.append(psnr_value)
            metric_4.append(rmse_sw_value)
            metric_5.append(psnrb_value)
            metric_6.append(sam_value)
            metric_7.append(msssim_value)
            metric_8.append(sam_value)
            

            
d_1 =  pd.DataFrame(metric_1)
d_2 =  pd.DataFrame(metric_2)
d_3 =  pd.DataFrame(metric_3)
d_4 =  pd.DataFrame(metric_4)
d_5 =  pd.DataFrame(metric_5)
d_6 =  pd.DataFrame(metric_6)
d_7 =  pd.DataFrame(metric_7)
d_8 =  pd.DataFrame(metric_8)
m_1 =  pd.DataFrame(upsampling_method)
m_2 =  pd.DataFrame(downsampling_method)
i_s =  pd.DataFrame(image_set)
i_m =  pd.DataFrame(image_scale)
i_n =  pd.DataFrame(image_number)


        
frames = [m_1,m_2,i_s,i_m,i_n,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8]
result = pd.concat(frames, axis=1)

            # Save the big LIST again as Dataframe and then save it as list_2.csv
df = pd.DataFrame(result)
print(df.shape)
print(d_4)
print(df)
df.to_csv('first_evaluation_test_2.csv', mode='a', header=False)

Folder:4_upscaled image number:0
Folder:4_upscaled image number:2
Folder:4_upscaled image number:12
(14, 13)
            0
0   31.722074
1   33.022681
2   33.022681
3   30.954153
4   30.823561
5   33.916186
6   29.605766
7   29.950692
8   32.558987
9   33.130295
10  31.692397
11  29.036801
12  29.036801
13  32.482327
          0        0      0  0   0           0          0          0  \
0   bicubic  bicubic  Set14  4   0         NaN        NaN        NaN   
1   bicubic  bicubic  Set14  4   1  587.927949  24.247226  20.437563   
2   bicubic  bicubic  Set14  4   2         NaN        NaN        NaN   
3   bicubic  bicubic  Set14  4   3  285.003467  16.882046  23.582302   
4   bicubic  bicubic  Set14  4   4  235.425629  15.343586  24.412266   
5   bicubic  bicubic  Set14  4   5  154.345134  12.423572  26.245874   
6   bicubic  bicubic  Set14  4   6  321.561325  17.932131  23.058166   
7   bicubic  bicubic  Set14  4   7  259.378577  16.105234  23.991463   
8   bicubic  bicubic  Set14  4   

In [139]:
print(d_4.mean(axis = 0))

0    31.496814
dtype: float64


In [140]:
print(d_3.mean(axis = 0))

0    24.761984
dtype: float64


In [187]:
# Read and evaluate
Directories = ['Set14','BSD100','Set5','Urban100','Celeba_HQ']
scales = [2]

import numpy as np
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio
from skimage.color import rgb2ycbcr
from skimage import img_as_float

from sewar.full_ref import mse, rmse, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb, psnr

Image_DataSet_List = []
Image_Scaled_List = []
Average_Metric_0_List = []
Average_Metric_1_List = []
Scale_List = []


for directory in Directories:  
    metric_name = []
    upsampling_method = []
    downsampling_method = []
    image_set = []
    image_scale = [] 
    image_number = []
    metric_1 = []
    metric_2 = []
    for scale in scales: 
        images_1 = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + "Original") 
        images_2 =  load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + "/HR/" + str(scale) + "_upscaled/")
        Scale_List.append(scale)   
        #images = load_images_from_folder('/Users/ege/Desktop/DataSets/' + directory + '/HR')
        for i, oriimg in enumerate(images_1): 
            
            pil_image_1 = images_1[i].convert('RGB') 
            pil_image_2 = images_2[i].convert('RGB')
            
            open_cv_image_1 = np.array(pil_image_1) 
            open_cv_image_2 = np.array(pil_image_2) 
            
            # Convert RGB to BGR 
            open_cv_image_1 = open_cv_image_1[:, :, ::-1].copy()
            open_cv_image_2 = open_cv_image_2[:, :, ::-1].copy()
            
            im1_t = np.atleast_3d(img_as_float(open_cv_image_1))
            im2_t = np.atleast_3d(img_as_float(open_cv_image_2))

            if im1_t.shape[2] == 1 or im2_t.shape[2] == 1:
                im1_t = im1_t[..., 0]
                im2_t = im2_t[..., 0]

            else:
                im1_t = rgb2ycbcr(im1_t)[:, :, 0:1] / 255.0
                im2_t = rgb2ycbcr(im2_t)[:, :, 0:1] / 255.0

            try: 
                psnr_val = peak_signal_noise_ratio(im1_t, im2_t)
            except:
                print('error at: ' + str(i))

            # Merge all the lists/Pandas Dataframes in one and then concatenate them as a LIST
            upsampling_method.append('bicubic')
            downsampling_method.append('bicubic')
            image_set.append(str(directory))
            image_scale.append(str(scale))
            image_number.append(str(i))
            metric_1.append(psnr_val)
            metric_2.append('*')
          
        
        
    d_1 =  pd.DataFrame(metric_1)
    d_2 =  pd.DataFrame(metric_2)
    m_1 =  pd.DataFrame(upsampling_method)
    m_2 =  pd.DataFrame(downsampling_method)
    i_s =  pd.DataFrame(image_set)
    i_m =  pd.DataFrame(image_scale)
    i_n =  pd.DataFrame(image_number)

    frames = [m_1,m_2,i_s,i_m,i_n,d_1,d_2]
    result = pd.concat(frames, axis=1)
    df = pd.DataFrame(result)
    
    print(df)
    
    Image_DataSet_List.append(directory)
    Average_Metric_0_List.append(d_1.mean())
    Average_Metric_1_List.append(d_2.mean())
   
    

I_D_L =  pd.DataFrame(Image_DataSet_List)
A_M_0 =  pd.DataFrame(Average_Metric_0_List)
A_M_1 =  pd.DataFrame(Average_Metric_1_List)
S_L   =  pd.DataFrame(Scale_List)
    
frames_overall = [I_D_L,A_M_0,A_M_1,S_L]
result_overall = pd.concat(frames_overall, axis=1)
df_overall= pd.DataFrame(result_overall)

print(df_overall)


          0        0      0  0   0          0  0
0   bicubic  bicubic  Set14  2   0  30.684806  *
1   bicubic  bicubic  Set14  2   1  26.585917  *
2   bicubic  bicubic  Set14  2   2  26.016277  *
3   bicubic  bicubic  Set14  2   3  27.942775  *
4   bicubic  bicubic  Set14  2   4  29.265910  *
5   bicubic  bicubic  Set14  2   5  32.931336  *
6   bicubic  bicubic  Set14  2   6  27.815216  *
7   bicubic  bicubic  Set14  2   7  29.133704  *
8   bicubic  bicubic  Set14  2   8  34.145069  *
9   bicubic  bicubic  Set14  2   9  32.717302  *
10  bicubic  bicubic  Set14  2  10  35.002439  *
11  bicubic  bicubic  Set14  2  11  24.530641  *
12  bicubic  bicubic  Set14  2  12  30.571258  *
13  bicubic  bicubic  Set14  2  13  33.257687  *
          0        0       0  0   0          0  0
0   bicubic  bicubic  BSD100  2   0  32.314055  *
1   bicubic  bicubic  BSD100  2   1  29.796060  *
2   bicubic  bicubic  BSD100  2   2  28.050072  *
3   bicubic  bicubic  BSD100  2   3  23.181740  *
4   bicubic  bi

0    26.014496
dtype: float64